# Image_Classification_Pytorch (Inference)

<a class="anchor" id="0"></a>
# Table of Contents

1. [套件安裝與載入](#1)
1. [環境檢測與設定](#2)
1. [開發參數設定](#3)
1. [資料處理](#4)
    -  [載入CSV檔](#4.1)
    -  [檢查CSV檔缺失值](#4.2)
1. [推論](#5)
1. [待辦事項](#6)

# 1. 套件安裝與載入<a class="anchor" id="1"></a>
[Back to Table of Contents](#0)

In [ ]:
!pip3 install git+https://github.com/rwightman/pytorch-image-models.git
# !pip3 install iterative-stratification

In [ ]:
# 資料處理套件
import os
import gc
import cv2
import sys
import time
import timm
import copy
import random
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import albumentations as A
import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm
from albumentations.pytorch.transforms import ToTensorV2
# from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import (roc_auc_score, confusion_matrix, accuracy_score, 
                             precision_score, recall_score, f1_score, classification_report, cohen_kappa_score)

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# 設定顯示中文字體
from matplotlib.font_manager import FontProperties
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei'] # 用來正常顯示中文標籤
plt.rcParams['font.family'] = 'AR PL UMing CN'
plt.rcParams['axes.unicode_minus'] = False # 用來正常顯示負號

In [ ]:
# pytorch深度學習模組套件
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F
import torchvision

from torch.optim import lr_scheduler
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision import models

# 2. 環境檢測與設定<a class="anchor" id="2"></a>
[Back to Table of Contents](#0)

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

In [ ]:
# 查看pytorch版本
print(torch.__version__)

In [ ]:
'''執行環境參數設定'''

# (Boolean)是否為本機
LOCAL = False

# (Boolean)是否為 Colab
COLAB = False


'''檔案路徑參數設定'''

# (String)Root路徑
if LOCAL:
    PATH = r'../'
elif COLAB:
    PATH = r'/content/drive/My Drive/Colab Notebooks/'
else:
    PATH = r'../input/'
    OUTPUT_PATH = r'/kaggle/working/'
    
# (String)資料根路徑
DATA_ROOT_PATH = PATH+r'resized-2015-2019-blindness-detection-images/' 

# (String)訓練資料路徑
TRAIN_DATA_PATH = DATA_ROOT_PATH+r'resized train 15/'

# (String)訓練CSV路徑
TRAIN_CSV_PATH = DATA_ROOT_PATH+r'labels/trainLabels15.csv'

# (String)專案名稱
PROJECT_NAME = 'aptos2019-blindness-detection'

# (Boolean)是否要匯入Library
IMPORT_PYTORCH_LIBRARY = False

# (String)Library的路徑
PYTORCH_LIBRARY_PATH = PATH + "PyTorch_Library/"

# (String)專案檔案儲存路徑
if LOCAL or COLAB:
    OUTPUT_PATH = PATH
PROJECT_PATH = OUTPUT_PATH+PROJECT_NAME+'/'+PROJECT_NAME+' '+datetime.datetime.now().strftime("%Y-%m-%d %H:%M")

# (String)權重名稱(使用哪個權重)
WEIGHTS_NAME = 'efficientnet_v2_b0'

# (String)模型名稱(使用哪個模型)
MODEL_NAME = 'efficientnet_v2_b0'

# (String)讀取預訓練權重的儲存路徑
LOAD_WEIGHTS_PATH = PROJECT_PATH+r'/models/pretrain_weights/'+WEIGHTS_NAME+'.pth'

# (String)讀取預訓練模型的儲存路徑
LOAD_MODEL_PATH = PROJECT_PATH+r'/models/pretrain_models/'+MODEL_NAME+'.pth'

# (Boolean)是否建立訓練模型儲存的時間戳資料夾
MODEL_TIME_FOLDER = False
# (String)訓練模型的儲存路徑
if MODEL_TIME_FOLDER:
    TRAIN_MODEL_PATH = PROJECT_PATH+r'/models/'+MODEL_NAME+'.pth'
else:
    TRAIN_MODEL_PATH = MODEL_NAME+'.pth'

In [ ]:
if DEVICE != torch.device("cpu"):
    !nvidia-smi

In [ ]:
if not LOCAL and COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    
if MODEL_TIME_FOLDER:
    if not os.path.isdir(PROJECT_PATH+r'/models/'):
        os.makedirs(PROJECT_PATH+r'/models/')
    
if IMPORT_PYTORCH_LIBRARY:
    sys.path.append(PYTORCH_LIBRARY_PATH + "Custom_Loss.py")
    sys.path.append(PYTORCH_LIBRARY_PATH + "Custom_Model.py")

# 3. 開發參數設定<a class="anchor" id="3"></a>
[Back to Table of Contents](#0)

In [ ]:
'''客製參數設定'''


'''資料參數設定'''

# (Int)分類數量
CLASSES = 5

# (Int)有CSV檔該參數才有用，1則為不做交叉驗證
FOLD = 1
    
# (Int)圖片尺寸
IMAGE_SIZE = [224]*FOLD

# (String)圖片副檔名
IMAGE_NAME_EXTENSION = '.jpg'

# (String)CSV圖片檔名欄位
IMAGE_NAME = 'image'

# (String)CSV標籤欄位
LABEL_NAME = 'level'

# (String)CSV標籤欄位類型
LABEL_NAME_TYPE = 'string'

# (Boolean)CSV圖片檔名欄位是否包含副檔名
IMAGE_NAME_HAVE_EXTENSION = False

# (Boolean)是否轉DataSet時，讓標籤做獨熱編碼
ONE_HOT_LABEL = False

#  (Boolean)圖像轉為RGB
COLOR_CONVERT_RGB = True

# (Int)不同的種子會產生不同的Random或分層K-FOLD分裂, 42則是預設固定種子
SEED = 42

if FOLD == 1:
    # (Float)驗證集佔訓練集的比率，FOLD>1則不啟用
    DATA_SPLIT = 0.2
else:
    # (String)切分訓練集跟驗證集方式 StratifiedKFold, MultilabelStratifiedKFold
    KF = StratifiedKFold(n_splits=FOLD, shuffle=True, random_state=SEED)

# (Boolean)是否資料轉Tensor時啟動鎖頁內存(GPU內存)，而不鎖頁內存就是會使用到硬碟虛擬內存
PIN_MEMORY = False

# (Int)要用於數據加載的子進程數。0表示將在主進程中加載數據。（默認值：0）
NUM_WORKERS = 0

# (Boolean)批次處理在大小不合適的情況下，是否刪除最後一個不完整的批次
DROP_LAST = False
    
# (Boolean)如為True每次返回的卷積算法將是確定的，即默認算法
CUDNN_DETERMINISTIC = True

# (Boolean)PyTorch 中對模型裡的卷積層進行預先的優化，也就是在每一個卷積層中測試 cuDNN 提供的所有卷積實現算法，
# 然後選擇最快的那個。這樣在模型啟動的時候，只要額外多花一點點預處理時間，就可以較大幅度地減少訓練時間
CUDNN_BENCHMARK = True


'''資料擴增參數設定

資料擴增教學
https://zhuanlan.zhihu.com/p/107399127

資料擴增Doc
https://vfdev-5-albumentations.readthedocs.io/en/docs_pytorch_fix/api/augmentations.html
'''

# (Float)訓練集資料擴增的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_TRAIN_TRANSFORMS = 1.0

# (Float)驗證集資料擴增的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_VAL_TRANSFORMS = 1.0

# 以下資料擴增為訓練集使用=============================================

# (Float)模糊的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_BLUR = 0

# (Int)模糊的上限
BLUR_LIMIT = 3

# (Float)水平翻轉的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_HORIZONTALFLIP = 0

# (Float)垂直翻轉的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_VERTICALFLIP = 0

# (Float)水平和垂直翻轉的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_FLIP = 0

# (Float)隨機旋轉90度的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_RANDOMROTATE90 = 0

# (Float)平移縮放旋轉的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_SHIFTSCALEROTATE = 0

# (Float)平移縮放旋轉的平移上限
SHIFTSCALEROTATE_SHIFT_LIMIT = 0.0625

# (Float)平移縮放旋轉的縮放上限
SHIFTSCALEROTATE_SCALE_LIMIT = 0.1

# (Float)平移縮放旋轉的旋轉上限
SHIFTSCALEROTATE_ROTATE_LIMIT = 45

# (Float)彈性變換的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_ELATICTRANSFORM = 0

# (Float)彈性變換的alpha高斯過濾參數
ELATICTRANSFORM_ALPHA = 1

# (Float)彈性變換的sigma高斯過濾參數
ELATICTRANSFORM_SIGMA = 50

# (Float)彈性變換的alpha_affine，範圍為（-alpha_affine，alpha_affine）
ELATICTRANSFORM_ALPHA_AFFINE = 50

# (Float)網格失真的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_GRIDDISTORTION = 0

# (Int)網格失真的每一條邊上網格單元數量
GRIDDISTORTION_NUM_STEPS = 5

# (Float)隨機亮度對比度的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_RANDOMBRIGHTNESSCONTRAST_CONTRAST = 0

# (Float)隨機亮度的上限
RANDOMBRIGHTNESSCONTRAST_BRIGHTNESS_LIMIT = 0.2

# (Float)隨機對比度的上限
RANDOMBRIGHTNESSCONTRAST_CONTRAST_LIMIT = 0.2

# (Float)隨機色調飽和度的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_HUESATURATIONVALUE = 0

# (Float)隨機色調飽和度的色調上限
HUESATURATIONVALUE_HUE_SHIFT_LIMIT = 20

# (Float)隨機色調飽和度的飽和度上限
HUESATURATIONVALUE_SAT_SHIFT_LIMIT = 30

# (Float)隨機色調飽和度的值上限
HUESATURATIONVALUE_VAL_SHIFT_LIMIT = 20

# (Float)對比度受限自適應直方圖均衡的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_CLAHE = 0

# (Float)對比度受限自適應直方圖均衡的對比度上限
CLAHE_CLIP_LIMIT = 4.0

# (Float)隨機在圖像上生成黑色矩形的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_COARSEDROPOUT = 0

# (Int)隨機在圖像上生成黑色矩形的數量
COARSEDROPOUT_NUM_HOLES = 8

# (Int)隨機在圖像上生成黑色矩形的最大高度
COARSEDROPOUT_MAX_H_SIZE = 8

# (Int)隨機在圖像上生成黑色矩形的最大寬度
COARSEDROPOUT_MAX_W_SIZE = 8

# (Float)隨機縮放剪裁的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_RANDOMRESIZEDCROP = 0

# (Float Tuple)隨機縮放剪裁之前的圖像比例縮放
RANDOMRESIZEDCROP_SCALE = (0.08, 1.0)

# (Int)隨機縮放剪裁之前的圖像高度
RANDOMRESIZEDCROP_HEIGHT = IMAGE_SIZE[0]

# (Int)隨機縮放剪裁之前的圖像寬度
RANDOMRESIZEDCROP_WIDTH = IMAGE_SIZE[0]

# 以上資料擴增為訓練集使用=============================================

# 以下資料擴增為訓練集和驗證集共用======================================

# (Float)縮放的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_RESIZE = 1.0

# (Int)縮放後的圖片高度
RESIZE_HEIGHT = IMAGE_SIZE[0]

# (Int)縮放後的圖片寬度
RESIZE_WIDTH = IMAGE_SIZE[0]

# (Float)正規化的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_NORMALIZE = 1.0

# (List)正規化的平均值([0,1]的參考平均值:[0.485, 0.456, 0.406], [-1,1]的參考平均值:[0.5, 0.5, 0.5])
NORMALIZE_MEAN = [0.485, 0.456, 0.406]

# (List)正規化的標準差([0,1]的參考標準差[0.229, 0.224, 0.225], [-1,1]的參考標準差[0.5, 0.5, 0.5])
NORMALIZE_STD = [0.229, 0.224, 0.225]

# (Float)正規化的PIXEL最大值(參考PIXEL最大值255.0)
NORMALIZE_MAX_PIXEL_VALUE = 255.0

# (Float)歸一化的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
# ToTensorV2()將[0, 255]的PIL.Image或[H, W, C]的numpy.ndarray數據，
# 轉換為形狀[C, H, W]的torch.FloadTensor，並歸一化。
P_TOTENSORV2 = 1.0

# 以上資料擴增為訓練集和驗證集共用======================================


''''模型參數設定'''

# (Boolean)是否依照設定路徑，載入完整客製(模型+權重)
LOAD_MODEL = False

# (Boolean)使用客制模型，None則使用基本模型
CUSTOM_MODEL = None

# (Boolean)是否依照設定路徑，已有客製模型，僅載入權重
LOAD_WEIGHTS = False

if CUSTOM_MODEL is None and not LOAD_MODEL:
    # (Boolean)使用基礎timm模型，如為False則使用基礎Pytorch模型
    USE_BASE_TIMM_MODEL = True
    if USE_BASE_TIMM_MODEL:
        # https://github.com/rwightman/pytorch-image-models#getting-started-documentation
        # (Model)建立timm模型
        BASE_MODEL = "tf_efficientnetv2_b0"
    else:
        # (Model)建立Pytorch模型
        BASE_MODEL = models.resnet18
        
    # (Boolean)是否使用基礎模型權重
    LOAD_BASE_WEIGHTS = True

    # (Boolean)基礎模型是否可訓練權重(不包括頂部網路層)
    BASE_MODEL_TRAINABLE = False
    
    # (Boolean)基礎模型是否使用分類層輸出，否則為全連接層
    CLASSIFIER_OUTPUT = True

    # (Float)Dropout比率
    DROPOUT = 

    # (Boolean)Bias偏移量
    BIAS = True

# (Boolean)是否印出完整模型
MODEL_PRINT = False


''''回調函數參數設定

學習率遞減
https://zhuanlan.zhihu.com/p/69411064

回調函數Doc
https://pytorch.org/docs/stable/optim.html

'''

# (Boolean)回調函數 ModelCheckpoint 是否啟用
CALLBACKS_CHECK_POINTER = False

# (Boolean)回調函數 EarlyStopping 是否啟用
CALLBACKS_EARLY_STOPPING = True

# (Boolean)回調函數 StepLR 是否啟用
CALLBACKS_STEPLR = True

# (Boolean)回調函數 ReduceLROnPlateau 是否啟用
CALLBACKS_REDUCELRONPLATEAU = False

# (Boolean)回調函數 CosineAnnealingWarmRestarts 是否啟用
CALLBACKS_COSINEANNEALINGWARMRESTAERS = False

# (Boolean)回調函數 TensorBoard 是否啟用(訓練epoch step指標計算有啟用才有用)
CALLBACKS_TENSOR_BOARD = False

# (String)回調函數監控數值(val_auc 僅限雙分類) val_acc/val_loss/val_auc
MONITOR = 'val_loss'

# (Boolean)回調函數 ModelCheckpoint 是否只儲存最佳 False
SAVE_BEST_ONLY = True

# (Boolean)回調函數 ModelCheckpoint 是否只儲存權重 True
SAVE_WEIGHTS_ONLY = True

# (Int)回調函數 EarlyStopping 沒有改善的時期數，之後訓練將停止 10
PATIENCE_ELS = 5

# (Int)學習率衰減的時間段，意思每過多少epoch會衰減
STEP_SIZE = 5

# (Float)學習率衰減的乘數 0.1
GAMMA = 0.1

# (String)最小，最大之一 在最小模式下，當監視的數量停止減少時，lr將減小； 在最大模式下，當監視的數量停止增加時，它將減少。 min
MODE = "min"

# (Float)學習率降低的因數。new_lr = lr *因子 0.1
FACTOR = 0.1

# (Int)沒有改善的時期數，此後學習率將降低。例如，如果 耐心= 2，那麼我們將忽略前兩個時期而沒有任何改善，
# 並且如果損失仍然沒有改善，則只會在第三個時期之後降低LR。 10
PATIENCE = 10 

# (Float)用於測量新的最佳閾值，僅關注重大變化。 1e-4
THRESHOLD = 1e-4 

# (String)rel，abs之一。在rel模式下，“ max”模式下的dynamic_threshold = best *（1 +閾值），在min模式下，
# dynamic_threshold = best *（1-threshold）。在絕對模式下，dynamic_threshold =最佳+ 最大模式下的閾值或最佳-最小模式下的閾值。 rel
THRESHOLD_MODE = "rel"

# (Int)減少lr後恢復正常運行之前要等待的時期數。 0 
COOLDOWN = 0

# (Float/List)標量或標量列表。所有參數組或每個組的學習率的下限。 0 
MIN_LR = 0

# (Float)應用於lr的最小衰減。如果新舊lr之間的差異小於eps，則忽略該更新。 1e-8
SCHEDULER_EPS = 1e-8

# (Int)第一次重啟的迭代次數。
T_0 = 15

# (Int)重新啟動後，因素增加。 1 
T_MULT = 1

# (Int)最低學習率。 0
ETA_MIN = 0

# (Boolean)每次更新，學習率回調函式都會向輸出印出一條消息。 False
SCHEDULER_VERBOSE = False

# (Boolean)訓練集每批就更新學習率回調函式，否則每時代就更新。 False
SCHEDULER_BATCH_UPDATE = False

# (Boolean)驗證集學習率回調函式是否啟用。 False
VAL_ENABLE_SCHEDULER = False

# (Boolean)驗證集通過計算LOSS就更新學習率回調函式，否則不計算就更新。 False
SCHEDULER_LOSS_UPDATE = False

# (String)TensorBoard儲存檔案的註解
TENSOR_BOARD_COMMENT = PROJECT_NAME


''''編譯參數設定

編譯參數Doc
https://pytorch.org/docs/stable/optim.html

'''

# (String)優化器指定(SGD/Adam/Adamax/RMSprop/Adagrad)
OPTIMIZERS_TYPE = "Adam"

# (Float)優化器學習率 1e-3/1e-1
LEARNING_RATE = 1e-3

# (Float)學習速率衰減 0
LR_DECAY = 0

# (Float)優化器權重衰減 5e-5/5e-4
WEIGHT_DECAY = 5e-5

# (Float)加速優化器在相關方向上前進，並抑制震盪 0.9
MOMENTUM = None

# (Tuple Float)用於計算梯度及其平方的移動平均值的係數 (0.9, 0.999)
BETAS = (0.9, 0.999)

# (Float)分母中添加的項，以提高數值穩定性 1e-8
EPS = 1e-8

# (Float)平滑常數 0.99
ALPHA = 0.99

# (Boolean)計算居中RMSProp，則通過估計其方差來對梯度進行歸一化 True
CENTERED = True

# (Float)阻尼動量 0
DAMPENING = 0

# (Boolean)啟用Nesterov動量 False
NESTEROV = False

# (Boolean)客制損失函數
CUSTOM_LOSSES = nn.CrossEntropyLoss()

# (Boolean)是否要重塑標籤張量
VIEW_LABEL_TENSOR = False
if VIEW_LABEL_TENSOR:
    # # (Int)計算損失函數的標籤VIEW_A 1
    LOSS_LABEL_VIEW_A = 1

    # # (Int)計算損失函數的標籤VIEW_B -1
    LOSS_LABEL_VIEW_B = -1

# (Dtype)訓練時輸入tensor dtype
INPUTS_TENSOR_DTYPE = torch.float

# (Dtype)訓練時標籤tensor dtype
LABELS_TENSOR_DTYPE = torch.long

# https://blog.csdn.net/jzlin1997/article/details/110048060
# (Boolean)是否進到Loss前的Ootput，要特別經過激活函數 False
USE_ACTIVITION_BEFORE_LOSS = False
if USE_ACTIVITION_BEFORE_LOSS:
    # Loss前Ootput的激活函數
    ACTIVITION_BEFORE_LOSS = nn.Sigmoid()

# (String )評價指標的圖表顯示 accuracy/auc
PLOT_METRICS = 'accuracy'

# (Boolean)是否印出完整編譯器
OPTIMIZER_PRINT = False


''''訓練參數設定'''

# (Boolean)是否啟用混合精度訓練
AMP_SCALE_TRAIN = True

# (Int List)每批訓練的尺寸
BATCH_SIZE = [256]*FOLD

# (Int)訓練做幾次時代
EPOCHS = [30]*FOLD

# (Int)指定列印進度條的位置（從0開始）
TQDM_POSITION = 0

# (Boolean)保留迭代結束時進度條的所有痕跡。如果是None，只會在position是0時離開
TQDM_LEAVE = True

# https://kozodoi.me/python/deep%20learning/pytorch/tutorial/2021/02/19/gradient-accumulation.html
# (Int)假設您要在一批中使用32張圖像，但是一旦超出8張，硬件就會崩潰
# 在這種情況下，您可以使用8張圖像的批次並每4批次更新一次權重，因此設置 4
ACCUM_ITER = 1

# (Boolean)是否計算每個Epoch Step指標，會造成CPU使用率變高
CALCULATE_EPOCH_STEP = False


''''圖表參數設定'''

# (Float)全部SNS圖表的字形縮放
ALL_SNS_FONT_SCALE = 1.0

# (Int)CSV缺失值圖表寬度
CSV_COUNTPLOT_FIGSIZE_W = 10

# (Int)CSV缺失值圖表高度
CSV_COUNTPLOT_FIGSIZE_H = 10

# (Int)CSV缺失值圖表標題字型大小
CSV_COUNTPLOT_TITLE_FONTSIZE = 20

# (Int)CSV缺失值圖表X軸標題字型大小
CSV_COUNTPLOT_XLABEL_FONTSIZE = 15

# (Int)CSV缺失值圖表Y軸標題字型大小
CSV_COUNTPLOT_YLABEL_FONTSIZE = 15

# (Int)訓練歷程圖表寬度
TRAINING_CURVES_FIGSIZE_W = 20

# (Int)訓練歷程圖表高度
TRAINING_CURVES_FIGSIZE_H = 10

# (Int)訓練歷程圖表SCATTER的標記點大小 
TRAINING_CURVES_SCATTER_SCALAR = 200

# (Float)訓練歷程圖表SCATTER的指標文字離標記點X距離係數
TRAINING_CURVES_SCATTER_METRICS_TEXT_XSCALAR = 0.03

# (Float)訓練歷程圖表SCATTER的指標文字標記點Y距離係數
TRAINING_CURVES_SCATTER_METRICS_TEXT_YSCALAR = 0.13

# (Float)訓練歷程圖表SCATTER的損失文字標記點X距離係數
TRAINING_CURVES_SCATTER_LOSS_TEXT_XSCALAR = 0.03

# (Float)訓練歷程圖表SCATTER的損失文字標記點Y距離係數
TRAINING_CURVES_SCATTER_LOSS_TEXT_YSCALAR = 0.05

# (Int)訓練歷程圖表SCATTER的文字大小
TRAINING_CURVES_SCATTER_TEXTSIZE = 15

# (Int)訓練歷程圖表X軸標題字型大小
TRAINING_CURVES_XLABEL_FONTSIZE = 15

# (Int)訓練歷程圖表Y軸標題字型大小
TRAINING_CURVES_YLABEL_FONTSIZE = 15

# (Int)訓練歷程圖表標題字型大小
TRAINING_CURVES_TITLE_FONTSIZE = 20

# (Float)訓練歷程圖表格線粗度
TRAINING_CURVES_GRID_ALPHA = 0

# (Int)混淆矩陣圖表寬度
CONFUSION_MATRIX_FIGSIZE_W = 10

# (Int)混淆矩陣圖表高度
CONFUSION_MATRIX_FIGSIZE_H = 10

# (Int)混淆矩陣圖表內容字型大小
CONFUSION_MATRIX_HEATMAP_FONTSIZE = 15

# (Int)混淆矩陣圖表標題字型大小
CONFUSION_MATRIX_TITLE_FONTSIZE = 20

# (Int)混淆矩陣圖表X軸標題字型大小
CONFUSION_MATRIX_XLABEL_FONTSIZE = 15

# (Int)混淆矩陣圖表Y軸標題字型大小
CONFUSION_MATRIX_YLABEL_FONTSIZE = 15

# (String)預設'binary'：僅在目標為二進制，兩分類時適用。
#'micro'：通過計算總的真陽性，假陰性和假陽性來全局計算指標，多分類時適用。
#'macro'：計算每個標籤的指標，並找到其未加權平均值。這沒有考慮標籤不平衡，多分類時適用。
AVERAGE = 'macro'

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = CUDNN_DETERMINISTIC
    torch.backends.cudnn.benchmark = CUDNN_BENCHMARK

seed_everything(SEED)

In [ ]:
# 設置sns圖表縮放係數
sns.set(font_scale = ALL_SNS_FONT_SCALE)

# 4. 資料處理<a class="anchor" id="4"></a>
[Back to Table of Contents](#0)

## 4.1 載入CSV檔 <a class="anchor" id="4.1"></a>
[Back to Table of Contents](#0)

In [ ]:
print('Reading data...')

# 讀取訓練資料集CSV檔
train_csv = pd.read_csv(TRAIN_CSV_PATH,encoding="utf8")

print('Reading data completed')

In [ ]:
# 顯示訓練資料集CSV檔
train_csv.head()

In [ ]:
print("Shape of train_data :", train_csv.shape)

## 4.2 檢查CSV檔缺失值 <a class="anchor" id="4.2"></a>
[Back to Table of Contents](#0)

In [ ]:
# 缺失值比率
total = train_csv.isnull().sum().sort_values(ascending = False)
percent = (train_csv.isnull().sum()/train_csv.isnull().count()*100).sort_values(ascending = False)
missing_train_csv  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_train_csv.head(missing_train_csv.shape[0])

In [ ]:
train_csv[LABEL_NAME].value_counts()

In [ ]:
f,ax = plt.subplots(figsize=(CSV_COUNTPLOT_FIGSIZE_W, CSV_COUNTPLOT_FIGSIZE_H))
sns.countplot(train_csv[LABEL_NAME], hue = train_csv[LABEL_NAME],ax = ax)
plt.title("LABEL COUNT", fontsize=CSV_COUNTPLOT_TITLE_FONTSIZE)
plt.xlabel(LABEL_NAME.upper(), fontsize=CSV_COUNTPLOT_XLABEL_FONTSIZE)
plt.ylabel("COUNT", fontsize=CSV_COUNTPLOT_YLABEL_FONTSIZE)
plt.legend()
plt.show()

# 5. 推論<a class="anchor" id="5"></a>
[Back to Table of Contents](#0)

# 6. 待辦事項<a class="anchor" id="6"></a>
[Back to Table of Contents](#0)